In [ ]:
import numpy as np
import scipy
from scipy import stats
from scipy.ndimage.filters import convolve
try:
    from astropy.io import fits
except ImportError:
    import pyfits as fits
import matplotlib.pyplot as plt
import pdb

def pad_psf(psf, spectrum):
    out_psf = np.zeros( spectrum.shape )
    start = len(spectrum)/2 - len(psf)/2
    end = start + len(psf)
    out_psf[start:end] = psf

    return out_psf

def rl_fft(raw_image, psf, niter, k=1, con_var=None):
    calc_chisq = lambda a, b, c, d: np.sum((a - b)**2 / (a + c)**2 / (d-1))
    
    conversion =  raw_image.mean() / 10
    raw_image /= conversion
    
    lucy = np.ones(raw_image.shape)
    ratio = k * np.ones(raw_image.shape)
    fft_psf = np.fft.fft(psf)
    
    con_var = sample_noise(raw_image)
    print ("using: ", con_var)

    norm = np.fft.ifft(np.fft.fft(ratio) * np.conj(fft_psf))
    fft_conv = fft_psf * np.fft.fft(lucy)
    lucy_conv = np.fft.ifft(fft_conv)

    chisq = calc_chisq(lucy_conv, raw_image, con_var, raw_image.size)
    print ("initial Chisq: {}".format(chisq))

    for iteration in range(niter):
        ratio = k * (raw_image + con_var) / (lucy_conv + con_var)
        fft_srat = np.fft.fft(ratio) * np.conj(fft_psf)

        lucy *= np.fft.ifft(fft_srat) / norm
        print (lucy.max(), lucy.mean(), lucy.min())
        fft_conv = fft_psf * np.fft.fft(lucy)
        lucy_conv = np.fft.ifft(fft_conv)
        size = lucy.size
        chisq = calc_chisq(lucy_conv, raw_image, con_var, raw_image.size)
        print ("Iteration {} Chisq: {}".format(iteration, chisq))
    lucy = lucy[range(size/2,size)+range(0,size/2)]
    return lucy * conversion

def richardson_lucy(image, psf, iterations=50, clip=True):
    #Richarson lucy method to calculate the deconvolution of two matrix
    #image represents the input image which is blurred and is needed to be deconvolved with the obtained filter
    #Iterations are necessary to get a close approximation
    direct_time = np.prod(image.shape + psf.shape)
    fft_time =  np.sum([n*np.log(n) for n in image.shape + psf.shape])
    time_ratio = 40.032 * fft_time / direct_time
    if time_ratio <= 1 or len(image.shape) > 2:      # Calculate the time required to calculate convolution 
        convolve_method = fftconvolve                # and selects more faster option  
    else:
        convolve_method = convolve
    image = image.astype(np.float)
    psf = psf.astype(np.float)
    im_deconv = 0.5 * np.ones(image.shape)           # Defining the deconvolved matrix which would be written over 
    psf_mirror = psf[::-1, ::-1]

    for _ in range(iterations):
        relative_blur = image / convolve_method(im_deconv, psf, 'same') #
        relative_blur[np.isnan(relative_blur)] = 0       # to prevent NaN from propogating throughout convulated matrix
        relative_blur[np.isnan(relative_blur)] = 0       # to prevent NaN from propogating throughout convulated matrix
        im_deconv *= convolve_method(relative_blur, psf_mirror, 'same')

    if clip:
        im_deconv[im_deconv > 1] = 1
        im_deconv[im_deconv < -1] = -1
    return im_deconv

def sample_noise( spectrum ):
    samples = [spectrum[start:start+300].std() for start in range(0, len(spectrum), 300) ]

    return np.median( samples )

def rl_damped(raw, psf, niter=2, damped=True, N=3, T=None, multiplier=1):

    conversion = raw.mean() / 10
    raw /= conversion
    lucy = np.ones(raw.shape) * raw.mean()
    for i in xrange(niter):
        if damped:
            print ("dampening")
            lucy_temp = convolve( lucy, psf, mode='mirror')
            ratio = dampen(lucy_temp, raw, N, T, multiplier)
        else:
            ratio = raw / convolve(lucy, psf, mode='mirror')

        ratio[ np.isnan(ratio) ] = 0

        top = convolve( ratio, psf, mode='mirror')
        top[ np.isnan(top) ] = 0

        lucy = lucy * (top / psf.sum())
        print ('iteration', i, lucy.mean(), raw.mean())

    return lucy * conversion
